<a href="https://colab.research.google.com/github/vivekverma1019/Handwritten-Word-Recognition/blob/master/Handwritten_Word_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### import libraries and mount Gdrive 

In [0]:
# import required libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import numpy as np
import random 
import os
import cv2
from google.colab.patches import cv2_imshow

In [14]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###### readfile() -->returns  (dict = {filename : gt}) given .txt file

In [0]:
def readfile(filename):
    res = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        for i in lines:
            res.append(i.strip())
        f.close()
    dic = {}
    for i in res:
        p = i.split(' ')
        dic[p[0]] = p[-1]
    return dic

###### split words.txt into train.txt and test.txt with 0.95 : 0.5

In [0]:
"""
trn = full[:int(len(full)*0.95)]
tst = full[int(len(full)*0.95):]
full = []
with open('/content/drive/My Drive/Downloads/iam_words/words.txt','r') as f:
    lines = f.readlines()
    lines = lines[18:]
    for i in lines:
        full.append(i.strip('\n'))
tr = open('/content/drive/My Drive/Downloads/iam_words/train.txt', 'w+')
for i in trn:
    tr.write(i + '\n')
tr.close()
ts = open('/content/drive/My Drive/Downloads/iam_words/test.txt', 'w+')
for i in tst:
    ts.write(i + '\n')
ts.close()
"""

"\ntrn = full[:int(len(full)*0.95)]\ntst = full[int(len(full)*0.95):]\nfull = []\nwith open('/content/drive/My Drive/Downloads/iam_words/words.txt','r') as f:\n    lines = f.readlines()\n    lines = lines[18:]\n    for i in lines:\n        full.append(i.strip('\n'))\ntr = open('/content/drive/My Drive/Downloads/iam_words/train.txt', 'w+')\nfor i in trn:\n    tr.write(i + '\n')\ntr.close()\nts = open('/content/drive/My Drive/Downloads/iam_words/test.txt', 'w+')\nfor i in tst:\n    ts.write(i + '\n')\nts.close()\n"

###### random_uniform_num():: get() : return shuffled indexes given batchsize

In [0]:
class random_uniform_num():
    def __init__(self, total):
        self.total = total
        self.range = [i for i in range(total)]
        np.random.shuffle(self.range)
        self.index = 0
    def get(self, batchsize):
        r_n=[]
        if(self.index + batchsize > self.total):
            r_n_1 = self.range[self.index:self.total]
            np.random.shuffle(self.range)
            self.index = (self.index + batchsize) - self.total
            r_n_2 = self.range[0:self.index]
            r_n.extend(r_n_1)
            r_n.extend(r_n_2)
        else:
            r_n = self.range[self.index : self.index + batchsize]
            self.index = self.index + batchsize
        return r_n

###### define charList = [chars] and char_dic={char : count}

In [0]:
charList = []
chars = " !\"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
for i in range(len(chars)):
    charList.append(chars[i])
char_dic = {}
for i,j in enumerate(charList):
    char_dic[j] = i

###### preprocess() : returns preprocessed image

In [0]:
def preprocess(img, imgSize, dataAugmentation=False):
	if img is None:
		img = np.zeros([imgSize[1], imgSize[0]])
	if dataAugmentation:
		stretch = (random.random() - 0.5) 
		wStretched = max(int(img.shape[1] * (1 + stretch)), 1)
		img = cv2.resize(img, (wStretched, img.shape[0]))
	(wt, ht) = imgSize
	(h, w) = img.shape
	fx = w / wt
	fy = h / ht
	f = max(fx, fy)
	newSize = (max(min(wt, int(w / f)), 1), max(min(ht, int(h / f)), 1))
	img = cv2.resize(img, newSize)
	target = np.ones([ht, wt]) * 255
	target[0:newSize[1], 0:newSize[0]] = img
	img = cv2.transpose(target)
	(m, s) = cv2.meanStdDev(img)
	m = m[0][0]
	s = s[0][0]
	img = img - m
	img = img / s if s>0 else img
	return img

###### gen() : mini-batch generator

In [0]:
def gen(data_file, image_path, batchsize=50, maxlabellength=32, imagesize=(128, 32)):
    image_label = readfile(data_file)
    _imagefile = [i for i, j in image_label.items()]
    x = np.zeros((batchsize,1, imagesize[0], imagesize[1]), dtype=np.float)
    labels = np.ones([batchsize, maxlabellength]) * 10000
    input_length = np.zeros([batchsize, 1])
    label_length = np.zeros([batchsize, 1])
    r_n = random_uniform_num(len(_imagefile))
    _imagefile = np.array(_imagefile)
    while 1:
        shufimagefile = _imagefile[r_n.get(batchsize)]
        for i, j in enumerate(shufimagefile):
            img1 = cv2.imread(os.path.join(image_path, j.split('-')[0] , '-'.join([j.split('-')[0],j.split('-')[1]]) ,j+'.png'),cv2.IMREAD_GRAYSCALE)
            img = preprocess(img1 , imagesize)
            x[i] = np.expand_dims(img, axis=0)
            str = image_label[j]
            if len(str) == 53:
                str = str[:32]
            label_length[i] = len(str)
            if(len(str) <= 0):
                print("len < 0", j)
            input_length[i] = 32
            labels[i, :len(str)] = [char_dic[k] for k in str]

        inputs = {'the_input': x,
                'the_labels': labels,
                'input_length': input_length,
                'label_length': label_length,
                }
        outputs = {'ctc': np.zeros([batchsize])}
        yield (inputs, outputs)

###### define basemodel

In [0]:
inputs = tf.keras.Input(shape=(1,128,32),name='the_input')

l1_cn = layers.Conv2D(filters=32,kernel_size=(5,5),strides=(1,1),data_format='channels_first',padding='same')(inputs)
l1_bn = layers.BatchNormalization(axis=1)(l1_cn)
l1_a = layers.Activation('relu')(l1_bn)
l1_mp = layers.MaxPooling2D(pool_size=(2,2) , padding="valid",data_format="channels_first")(l1_a)

l2_cn = layers.Conv2D(filters=64,kernel_size=(5,5),strides=(1,1),data_format="channels_first",padding='same')(l1_mp)
l2_bn = layers.BatchNormalization(axis=1)(l2_cn)
l2_a = layers.Activation('relu')(l2_bn)
l2_mp = layers.MaxPooling2D(pool_size=(2,2) , padding="valid",data_format="channels_first")(l2_a)

l3_cn = layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),data_format="channels_first",padding='same')(l2_mp)
l3_bn = layers.BatchNormalization(axis=1)(l3_cn)
l3_a = layers.Activation('relu')(l3_bn)
l3_mp = layers.MaxPooling2D(pool_size=(1,2) , padding="valid",data_format="channels_first")(l3_a)

l4_cn = layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),data_format="channels_first",padding='same')(l3_mp)
l4_bn = layers.BatchNormalization(axis=1)(l4_cn)
l4_a = layers.Activation('relu')(l4_bn)
l4_mp = layers.MaxPooling2D(pool_size=(1,2) , padding="valid",data_format="channels_first")(l4_a)

l5_cn = layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),data_format="channels_first",padding='same')(l4_mp)
l5_bn = layers.BatchNormalization(axis=1)(l5_cn)
l5_a = layers.Activation('relu')(l5_bn)
l5_mp = layers.MaxPooling2D(pool_size=(1,2) , padding="valid",data_format="channels_first")(l5_a)

r_s = layers.Permute((2,1))(tf.squeeze(l5_mp,axis=3))
l6_bl = layers.Bidirectional(layer=layers.LSTM(units=256,return_sequences=True), merge_mode='concat')(r_s)

l7_bl = layers.Bidirectional(layer=layers.LSTM(units=256,return_sequences=True), merge_mode='concat')(l6_bl)

l8_cn = layers.Conv2D(filters = 80,kernel_size=(1,1),data_format='channels_last',padding='same',kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.1))(tf.expand_dims(l7_bl,2))
l8_sf = layers.Activation("softmax")(l8_cn)
y_pred = tf.squeeze(l8_sf,axis=2)

basemodel = tf.keras.Model(inputs = inputs,outputs = y_pred)


###### define ctc lambda function for ctc loss calculation
###### define model
###### compile model

In [0]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
labels = tf.keras.Input(name='the_labels', shape=[None], dtype='float32')
input_length = tf.keras.Input(name='input_length', shape=[1], dtype='int64')
label_length = tf.keras.Input(name='label_length', shape=[1], dtype='int64')

loss_out = tf.keras.layers.Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])
model = tf.keras.Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='adam', metrics=['accuracy'])

###### training

In [0]:
batch_size = 50
maxlabellength = 32
imagesize = (128,32)

In [0]:
train_loader = gen('/content/drive/My Drive/Downloads/iam_words/train.txt', '/content/drive/My Drive/Downloads/iam_words/words', batchsize=batch_size, maxlabellength=maxlabellength, imagesize=imagesize)
test_loader = gen('/content/drive/My Drive/Downloads/iam_words/test.txt', '/content/drive/My Drive/Downloads/iam_words/words', batchsize=batch_size, maxlabellength=maxlabellength, imagesize=imagesize)

In [0]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/My Drive/models/weights-{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)

In [0]:
model.fit_generator(train_loader,
    	steps_per_epoch = 109554//batch_size,
    	epochs = 10,
    	initial_epoch = 0,
    	validation_data = test_loader,
    	validation_steps = 5766//batch_size,
        callbacks = [checkpoint])